In [2]:
# authenticate your kaggle api key
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [6]:
# Extract

api.dataset_download_files("yashgupta24/48000-movies-dataset", "./data/48000_movies/", force=True, unzip=True)
api.dataset_download_files("rounakbanik/the-movies-dataset", "./data/the-movies-dataset/", force=True, unzip=True)